In [ ]:
!pip uninstall splinter

In [4]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import requests
import pprint
from itertools import chain

ModuleNotFoundError: No module named 'splinter'

In [ ]:
def main():
    
    #get_brewery_api()
    scrape_untapped()


def get_brewery_api():
    i = 0
    url = f'https://api.openbrewerydb.org/breweries?page={i}&per_page=200'
    brewery_list = []
    for i in range(1,44):
        response = requests.get(url)
        brewery_data = response.json()
        brewery_list.append(brewery_data)
        time.sleep(1)
    all_brewerys_pd = list(chain.from_iterable(brewery_list))
    return all_brewerys_pd


def scrape_untapped():
    '''Scrape the untapped website to collect the latest beer information
    open the website'''
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=False)
    #reading the html code
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    brew_types = ['macro_brewery', 'micro_brewery', 'nano_brewery', 'brew_pub','cidery','meadery',
              'contract_brewery', 'regional_brewery']

    all_brewery_info = []

    for types in brew_types:
        url_untapped = f'https://untappd.com/brewery/top_rated?brewery_type={types}'
        browser.visit(url_untapped)
        time.sleep(5)
        breweries = browser.find_by_xpath('//*[@id="slide"]/div[3]/div/div[1]/div/div[4]')
        for brewery in breweries:
            title_link = brewery.text
            all_brewery_info.append(title_link)
            time.sleep(1)
    #Removing the adds        
    for item in all_brewery_info:
        if item == "Report":
            all_brewery_info.remove("Report Ad")

    brewery_list_all = []

    brewery_list = all_brewery_info[0].split("\n")
    brewery_list.remove("Report Ad")
    i = 0

    while i < len(brewery_list):
#         if i == "Report Ad":
#             brewery_list.remove("Report Ad")
#         else:
        try: 
            brewery = brewery_list[i:i+5]
            #print(brewery)
            dictionary = dict()
            dictionary['brewery_name'] = brewery[0]
            dictionary['brewery_location'] = brewery[1]
            dictionary['brewery_type'] = brewery[2]
            dictionary['number_beers'] = brewery[3]
            dictionary['brewery_ratings'] = brewery[4]
        except IndexError:
            print(f"This batch broke: {brewery}")
            break
        brewery_list_all.append(dictionary)
        i += 5
    return brewery_list_all

    
    # return dict_data
    if __name__ == "__main__":
        main()